<a href="https://colab.research.google.com/github/zoistath/Biosignal-assignment-2024-2025-/blob/main/Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Load and unzip data


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip drive/MyDrive/both.zip

Archive:  drive/MyDrive/both.zip
   creating: both/
  inflating: both/pilot21_evnt2_session1_filtered.csv  
  inflating: both/pilot17_evnt13_session1_filtered.csv  
  inflating: both/pilot16_evnt1_session1_filtered.csv  
  inflating: both/pilot16_evnt2_session1_filtered.csv  
  inflating: both/pilot12_evnt16_session1_filtered.csv  
  inflating: both/pilot1_evnt6_session1_filtered.csv  
  inflating: both/pilot16_evnt13_session1_filtered.csv  
  inflating: both/pilot22_evnt17_session1_filtered.csv  
  inflating: both/pilot24_evnt13_session1_filtered.csv  
  inflating: both/pilot19_evnt13_session1_filtered.csv  
  inflating: both/pilot5_evnt15_session1_filtered.csv  
  inflating: both/pilot25_evnt14_session1_filtered.csv  
  inflating: both/pilot11_evnt14_session1_filtered.csv  
  inflating: both/pilot4_evnt15_session1_filtered.csv  
  inflating: both/pilot2_evnt4_session1_filtered.csv  
  inflating: both/pilot20_evnt19_session1_filtered.csv  
  inflating: both/pilot2_evnt11_session1_filt

### Imports

In [ ]:
!pip install BaselineRemoval

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from BaselineRemoval import BaselineRemoval
from scipy.signal import detrend,lfilter,firwin

### Preprocessing

In [ ]:
base_dir = "drive/MyDrive/Real"
print(os.listdir(base_dir))
for folder in os.listdir(base_dir):
    folder_path = os.path.join(base_dir, folder)
    if os.path.isdir(folder_path):
        print(f"Processing folder: {folder}")
        for file in sorted(os.listdir(folder_path)):
            print(file)
            if file.startswith('pilot10') and file.endswith('.csv'):
              df = pd.read_csv(os.path.join(folder_path, file), names=None, header=None)
              channel_names = ['Fp1', 'AF7', 'AF3', 'F1', 'F3', 'F5', 'F7', 'FT7', 'FC5', 'FC3', 'FC1', 'C1', 'C3', 'C5', 'T7', 'TP7', 'CP5', 'CP3', 'CP1', 'P1', 'P3', 'P5', 'P7', 'P9', 'PO7', 'PO3', 'O1', 'Oz', 'POz', 'Pz', 'CPz', 'Fpz', 'Fp2', 'AF8', 'AF4', 'AFz', 'Fz', 'F2', 'F4', 'F6', 'F8', 'FT8', 'FC6', 'FC4', 'FC2', 'FCz', 'Cz', 'C2', 'C4', 'C6', 'T8', 'TP8', 'CP6', 'CP4', 'CP2', 'P2', 'P4', 'P6', 'P8', 'P10', 'PO8', 'PO4', 'O2']
              df.index = channel_names
              raw=df.values

              #Baseline Removal
              corrected_data=[]
              for channel in raw:
                baseObj = BaselineRemoval(channel.tolist())

                # Apply IModPoly
                modpoly_output = baseObj.IModPoly(2)

                # Append list
                corrected_data.append(modpoly_output)

              corrected_data = np.array(corrected_data)

              #FIR filter
              fir_coefficients = firwin(21, [1, 35], pass_zero=False, fs=256)
              filtered_data = lfilter(fir_coefficients, 1, corrected_data, axis=1)

              #Mean removal and detrending
              filtered_data -= filtered_data.mean(axis=1, keepdims=True)  # Mean removal
              filtered_data = detrend(filtered_data, axis=1)  # Detrending


              #Save data
              processed_df = pd.DataFrame(filtered_data, index=channel_names)
              processed_df.to_csv((folder + "/"+file).replace('noisy','filtered'), index=False)

###Save to zips

In [ ]:
!zip -r drive/MyDrive/both.zip both

In [ ]:
!zip -r drive/MyDrive/none.zip none